In [15]:
from simulated_bifurcation import Ising
from tqdm import tqdm
import matplotlib.pyplot as plt
import pulp as pl
import numpy as np
import time
import os

N_range = [10,20,30,50,70,100,150,200]
scale_range = [6,5,4,3,2,1,0,-1]
N_samples = 100

def generate_J_h(N_range,scale_range,N_samples):
    for N in N_range:
        for scale in scale_range:
            for i in range(N_samples):
                J = np.random.uniform(-10,10,size=(N, N))/(10**scale)
                J = (J + J.T)/2
                J = J - np.diag(np.diag(J))
                h = np.random.uniform(-10,10,size=N)/(10**(scale-int(i%5)+2))

                with open('test_cbc/test_CBC_J_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'wb') as f:
                    np.save(f, J)

                with open('test_cbc/test_CBC_h_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'wb') as f:
                    np.save(f, h)

                with open('test_cbc/test_SB_J_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'wb') as f:
                    np.save(f, J/4)

                with open('test_cbc/test_SB_h_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'wb') as f:
                    np.save(f, h/2 - sum([J[:,i]/8 for i in range(N)]) - sum([J[i,:]/8 for i in range(N)]))

generate_J_h(N_range,scale_range,N_samples)

In [2]:
def create_optimize_cbc(J,h,J_SB,h_SB,max_seconds=-1):
    time_1 = time.time()
    size = J.shape[0]

    model = pl.LpProblem("Linear Optimization", pl.LpMinimize)
    x = pl.LpVariable.dicts('x', list(range(size)), lowBound=0, upBound=1, cat='Integer')
    z = pl.LpVariable.dicts('z', [(i,j) for i in range(size) for j in range(size)], lowBound=0, upBound=1, cat='Integer')

    for i in range(size):
        for j in range(size):
            model += z[i,j] <= x[i]
            model += z[i,j] <= x[j]
            model += z[i,j] >= x[i] + x[j] -1

    model += pl.lpSum([(-1/2)*J[i][j]*z[i,j] for i in range(size) for j in range(size)]) + pl.lpSum([x[i]*h[i] for i in range(size)])

    if max_seconds == -1:
        model.solve(pl.PULP_CBC_CMD(msg=0, timeLimit=60*5))
    else:
        model.solve(pl.PULP_CBC_CMD(msg=0, timeLimit=max_seconds))

    spins = [1 if x[i].varValue >= 0.99 else -1 for i in range(size)]
    energy = (-1/2)*sum([sum([J_SB[i,j]*spins[j] for j in range(size)])*spins[i] for i in range(size)]) + sum([h_SB[i]*spins[i] for i in range(size)])[0]

    processing_time = time.time() - time_1
    return (processing_time,np.array(spins),energy)

In [3]:
def create_optimize_SB(J,h):
    time_1 = time.time()

    model = Ising(J,h)
    model.optimize(convergence_threshold = 35,
        sampling_period = 60,
        time_step = 0.01, 
        symplectic_parameter = 2,
        agents = 5,
        ballistic=False,
        heated=True,
        final_pressure=1.,
        pressure_slope=0.2)

    processing_time = time.time() - time_1
    return(processing_time, model.ground_state, model.energy)

In [4]:
def test_protocol(N_range,scale_range,N_samples,compute_SB=True):
    for N in tqdm(N_range):
        for scale in scale_range:
            for i in range(N_samples):
                with open('test_cbc/test_SB_J_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'rb') as f:
                    test_SB_J = np.load(f)

                with open('test_cbc/test_SB_h_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'rb') as f:
                    test_SB_h = np.expand_dims(np.load(f),axis=1)

                with open('test_cbc/test_CBC_J_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'rb') as f:
                    test_CBC_J = np.load(f)

                with open('test_cbc/test_CBC_h_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'rb') as f:
                    test_CBC_h = np.load(f)

                processing_time_cbc, spins_cbc, energy_cbc = create_optimize_cbc(test_CBC_J,test_CBC_h,test_SB_J,test_SB_h)
                with open('test_cbc/results_CBC_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npz', 'wb') as f:
                    np.savez(f, processing_time=processing_time_cbc, spins=spins_cbc, energy=energy_cbc, J=test_SB_J, h=test_SB_h)

                if compute_SB:
                    processing_time_sb, spins_sb, energy_sb = create_optimize_SB(test_SB_J,test_SB_h)
                    processing_time_cbc_stopped, spins_cbc_stopped, energy_cbc_stopped = create_optimize_cbc(test_CBC_J,test_CBC_h,test_SB_J,test_SB_h,processing_time_sb)

                    with open('test_cbc/results_SB_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npz', 'wb') as f:
                        np.savez(f, processing_time=processing_time_sb, spins=spins_sb, energy=energy_sb, J=test_SB_J, h=test_SB_h)

                    with open('test_cbc/results_CBCs_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npz', 'wb') as f:
                        np.savez(f, processing_time=processing_time_cbc_stopped, spins=spins_cbc_stopped, energy=energy_cbc_stopped, J=test_SB_J, h=test_SB_h)
                
                    os.remove('test_cbc/test_SB_J_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy')
                    os.remove('test_cbc/test_SB_h_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy')
                    os.remove('test_cbc/test_CBC_J_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy')
                    os.remove('test_cbc/test_CBC_h_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy')

In [18]:
test_protocol(N_range[:1],scale_range,N_samples,compute_SB=False)

100%|██████████| 1/1 [00:29<00:00, 29.76s/it]


In [19]:
N = 10
scale = 1
i = 1

with open('test_cbc/results_CBC_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npz', 'rb') as f:
    data_CBC = np.load(f)
    print(data_CBC['J'])

"""
with open('test_cbc/results_SB_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npz', 'rb') as f:
    data_SB = np.load(f)

with open('test_cbc/results_CBCs_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npz', 'rb') as f:
    data_CBC_stopped = np.load(f)
"""

[[ 0.         -0.00886002 -0.01404931  0.00288673  0.02348764  0.00999745
  -0.003204    0.00184856 -0.01194099  0.0016838 ]
 [-0.00886002  0.          0.00911726 -0.01401998  0.00433422 -0.0095467
   0.00142408 -0.01016181  0.01782817 -0.00556043]
 [-0.01404931  0.00911726  0.          0.01875427 -0.00905596 -0.00932419
  -0.01255367  0.00377574 -0.01634557  0.00573375]
 [ 0.00288673 -0.01401998  0.01875427  0.         -0.00255743 -0.01353691
  -0.00421798  0.01110312 -0.02297656 -0.00267073]
 [ 0.02348764  0.00433422 -0.00905596 -0.00255743  0.          0.00253662
   0.00728224  0.00155693 -0.00283712 -0.00955113]
 [ 0.00999745 -0.0095467  -0.00932419 -0.01353691  0.00253662  0.
  -0.00033747 -0.00567702 -0.00323433 -0.01984821]
 [-0.003204    0.00142408 -0.01255367 -0.00421798  0.00728224 -0.00033747
   0.          0.01499336  0.00220932  0.00832736]
 [ 0.00184856 -0.01016181  0.00377574  0.01110312  0.00155693 -0.00567702
   0.01499336  0.         -0.00971826  0.00459762]
 [-0.0119

'\nwith open(\'test_cbc/results_SB_\' + str(N) + \'_\' + str(scale) + "_" + str(i) + \'.npz\', \'rb\') as f:\n    data_SB = np.load(f)\n\nwith open(\'test_cbc/results_CBCs_\' + str(N) + \'_\' + str(scale) + "_" + str(i) + \'.npz\', \'rb\') as f:\n    data_CBC_stopped = np.load(f)\n'